In [1]:
import pandas as pd
from time import time, sleep
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import json
import logging
from kiteconnect import KiteTicker

In [3]:
api_key = "f5ean4aaxq3wv9n3"
access_token = 'X3q3FdmWsSB34MzmkrBtj3xWc9rDxguU'

In [4]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '3minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=31, hour=15 , minute=30 , second=0)

tokens = [14956290]

In [5]:
candle = 1
first_candle_timestamp = datetime(2020, 8, 31, 10, 8)
candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
latest_candle = pd.DataFrame(columns=['price'])

In [6]:
# Callback for successful connection.
def on_connect(ws, response):
    logging.debug("on connect: {}".format(response))
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)

def on_close(ws, code, reason):
    logging.error("closed connection on close: {} {}".format(code, reason))

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))
    
def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

In [7]:
def on_order_update(ws, data):
    print("order update: ", data)

In [8]:
def on_ticks(ws, tick):
    print(datetime.strftime(tick[0]['timestamp'], "%Y-%m-%d %H:%M:%S")+"==> "+str(tick[0]['last_price']))
    candle+=1
    print(candle)

In [8]:
# Callback for tick reception.
def on_ticks(ws, ticks):
    #static data
    candle = 3
    
    current_timestamp = tick[0]['timestamp']
    current_timestamp_string = datetime.strftime(tick[0]['timestamp'], "%Y-%m-%d %H:%M:%S")
    latest_price = ticks[0]['last_price']
    print(latest_price)
    first_candle_timestamp = datetime(current_timestamp.year, current_timestamp.month, current_timestamp.day, current_timestamp.hour, (current_timestamp.minute//candle)*candle)
    
    if datetime.now().minute>=first_candle_timestamp.minute and datetime.now().minute<(first_candle_timestamp+timedelta(minutes=candle)).minute:
        latest_candle.loc[len(latest_candle)] = latest_price
    elif (first_candle_timestamp+timedelta(minutes=candle)).minute==0 and datetime.now().hour<(first_candle_timestamp+timedelta(minutes=candle)).hour:
        latest_candle.loc[len(latest_candle)] = latest_price
    else:
        price_list = list(latest_candle['price'])
        if price_list[0]<price_list[-1]:
            color = 'green'
        else:
            color = 'red'
        candle_data.loc[len(candle_data)] = [first_candle_timestamp, price_list[0], max(price_list), min(price_list), price_list[-1], color]
        first_candle_timestamp = first_candle_timestamp+timedelta(minutes=candle)
        latest_candle = pd.DataFrame(columns=['price'])
        print(candle_data)

In [9]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Initialise.
kws = KiteTicker(api_key, access_token, debug=True)
print("Tokens length", len(tokens))

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close
kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

# kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
# kws.connect(disable_ssl_verification=True)
kws.connect()

2020-08-31 10:57:47,971 - kiteconnect.ticker - DEBUG - Start WebSocket connection.


Tokens length 1
2020-08-31 10:57:47+0530 [-] Log opened.


2020-08-31 10:57:48,175 - root - DEBUG - on connect: {"peer": "tcp4:13.127.120.33:443", "headers": {"date": "Mon, 31 Aug 2020 05:27:48 GMT", "connection": "upgrade", "upgrade": "websocket", "sec-websocket-accept": "NJZW4sSY7xMbg5S7jyEJ80cOOvw="}, "version": 18, "protocol": null, "extensions": []}
2020-08-31 10:57:48,178 - kiteconnect.ticker - DEBUG - ping => 
2020-08-31 10:57:48,206 - kiteconnect.ticker - DEBUG - pong => b''


2020-08-31 10:57:48+0530 [-] 2020-08-31 10:57:48==> 24539.95
2020-08-31 10:57:48+0530 [KiteTickerClientProtocol (TLSMemoryBIOProtocol),client] Unhandled Error
	Traceback (most recent call last):
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\log.py", line 103, in callWithLogger
	    return callWithContext({"system": lp}, func, *args, **kw)
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\log.py", line 86, in callWithContext
	    return context.call({ILogContext: newCtx}, func, *args, **kw)
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\context.py", line 122, in callWithContext
	    return self.currentContext().callWithContext(ctx, func, *args, **kw)
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\context.py", line 85, in callWithContext
	    return func(*args,**kw)
	--- <exception caught here> ---
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\internet\selectreactor.py", line 149, in _doReadOrWrit

2020-08-31 10:57:48,636 - kiteconnect.ticker - ERROR - Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
2020-08-31 10:57:48,637 - root - ERROR - closed connection on error: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
2020-08-31 10:57:48,640 - kiteconnect.ticker - ERROR - Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
2020-08-31 10:57:48,643 - root - ERROR - closed connection on close: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


...

2020-08-31 10:57:55+0530 [KiteTickerClientProtocol (TLSMemoryBIOProtocol),client] <twisted.internet.tcp.Connector instance at 0x1eb74c65c08 disconnected IPv4Address(type='TCP', host='ws.kite.trade', port=443)> will retry in 2 seconds
2020-08-31 10:57:55+0530 [-] Stopping factory <kiteconnect.ticker.KiteTickerClientFactory object at 0x000001EB74C86E08>
2020-08-31 10:57:58+0530 [-] Starting factory <kiteconnect.ticker.KiteTickerClientFactory object at 0x000001EB74C86E08>


2020-08-31 10:57:58,335 - root - DEBUG - on connect: {"peer": "tcp4:13.127.120.33:443", "headers": {"date": "Mon, 31 Aug 2020 05:27:58 GMT", "connection": "upgrade", "upgrade": "websocket", "sec-websocket-accept": "eND87JRoGDdyUF6Sol/R7FhdlUg="}, "version": 18, "protocol": null, "extensions": []}
2020-08-31 10:57:58,341 - kiteconnect.ticker - DEBUG - ping => 
2020-08-31 10:57:58,344 - kiteconnect.ticker - DEBUG - Resubscribe and set mode: full - [14956290]
2020-08-31 10:57:58,374 - kiteconnect.ticker - DEBUG - pong => b''


2020-08-31 10:57:58+0530 [-] Received SIGINT, shutting down.
2020-08-31 10:57:58+0530 [-] 2020-08-31 10:57:58==> 24544.2
2020-08-31 10:57:58+0530 [KiteTickerClientProtocol (TLSMemoryBIOProtocol),client] Unhandled Error
	Traceback (most recent call last):
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\log.py", line 103, in callWithLogger
	    return callWithContext({"system": lp}, func, *args, **kw)
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\log.py", line 86, in callWithContext
	    return context.call({ILogContext: newCtx}, func, *args, **kw)
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\context.py", line 122, in callWithContext
	    return self.currentContext().callWithContext(ctx, func, *args, **kw)
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\twisted\python\context.py", line 85, in callWithContext
	    return func(*args,**kw)
	--- <exception caught here> ---
	  File "C:\Users\Admin\Anaconda3\lib\site-packages\tw

2020-08-31 10:57:58,817 - kiteconnect.ticker - ERROR - Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
2020-08-31 10:57:58,818 - root - ERROR - closed connection on error: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
2020-08-31 10:57:58,820 - kiteconnect.ticker - ERROR - Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
2020-08-31 10:57:58,822 - root - ERROR - closed connection on close: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


2020-08-31 10:57:58+0530 [KiteTickerClientProtocol (TLSMemoryBIOProtocol),client] <twisted.internet.tcp.Connector instance at 0x1eb74c65c08 disconnected IPv4Address(type='TCP', host='ws.kite.trade', port=443)> will retry in 2 seconds
2020-08-31 10:57:58+0530 [-] Stopping factory <kiteconnect.ticker.KiteTickerClientFactory object at 0x000001EB74C86E08>
2020-08-31 10:57:58+0530 [-] Main loop terminated.


In [14]:
import json
import requests
import time

In [15]:
def main():
    r = requests.get("https://api.kite.trade/quote/ltp?api_key=f5ean4aaxq3wv9n3&access_token=bWjEfQWLA8YSRoO1UWu3i4bBfgTPnf1I&i=NFO:BANKNIFTY20AUGFUT")
    r2 = r.json()
    print(r2)
    #price = r2['data']['last_price']
    #timestamp = r2['data']['last_time']
    #print(timestamp+"==>"+price)
    #return price, timestamp

In [16]:
while True:
    main()
    #print(timestamp+"==>"+price)
    time.sleep(1)

{'status': 'success', 'data': {'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226, 'last_price': 23441.35}}}
{'status': 'success', 'data': {'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226, 'last_price': 23441.35}}}
{'status': 'success', 'data': {'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226, 'last_price': 23441.35}}}
{'status': 'success', 'data': {'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226, 'last_price': 23441.35}}}
{'status': 'success', 'data': {'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226, 'last_price': 23441.35}}}
{'status': 'success', 'data': {'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226, 'last_price': 23441.35}}}


KeyboardInterrupt: 